<a href="https://www.kaggle.com/code/shadym0hamed/fire-and-accident-detection?scriptVersionId=119679439" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# 1- Importing Libraries


In [ ]:
import matplotlib.pylab as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers
from tensorflow.keras.models import Sequential
import cv2
from keras.preprocessing.image import ImageDataGenerator

# 2- Importing Dataset

In [ ]:
data_dir = 'Dataset'

import pathlib
data_dir = pathlib.Path(data_dir)
list(data_dir.glob('*/*.jpg'))

image_count = len(list(data_dir.glob('*/*.jpg')))
objects = {
'Adeno' : list(data_dir.glob('Adeno/*')),
'Carcinoma' : list(data_dir.glob('Carcinoma/*')),
'Normal' : list(data_dir.glob('Normal/*'))
}

# 3- Assigning Binary Values to each class of the dataset

In [ ]:
objects_labels = {
    'Adeno' : 0,
   'Carcinoma' : 1,
   'Normal' : 2
    }
print(image_count)

# 4- Saving Dataset in variables to be able to train them.

In [ ]:
X, y = [], []

for name, images in objects.items():
    print("Printing...")
    for image in images:
          
        img = cv2.imread(str(image))
        
      
        try:
            resized = cv2.resize(img, (128,128))
            X.append(resized)
            y.append(objects_labels[name])
        except:
            print('skipped')
            break
        
X = np.array(X)
y = np.array(y)

# 5- Splitting the data to training and testing

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1,random_state = 0)

print(len(X_train), len(X_test),len(y_train), len(y_test))

X_train_scaled = X_train / 255
X_test_scaled = X_test / 255

# 6- Builing the CNN model

In [ ]:
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), padding='same', activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    
    layers.Conv2D(32, (3,3), padding='same', activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    
    
    
    layers.Conv2D(64, (3,3), padding='same', activation='relu'),

    layers.Conv2D(250, (3,3), padding='same', activation='relu'),
    layers.Conv2D(128, (3,3), padding='same', activation='relu'),
    layers.AvgPool2D(2, 2),
    
    layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    layers.AvgPool2D(2, 2),
    
    layers.Conv2D(256, (2, 2), padding="same", activation="relu"),
    layers.MaxPooling2D(2, 2),
    


    layers.Flatten(),
    layers.Dense(32,  activation='relu'),
    layers.Dropout(0.25),
    layers.Dense(3, activation='softmax'),
       
    ])

model.compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
    )

# 7- Training the model

In [ ]:
hitory = model.fit(X_train_scaled, y_train, epochs = 10)

# 8- Saving the model to an H5 file

In [ ]:
model.save('model2.h5')